In [1]:
import tushare as ts
from pyecharts.faker import Faker
from pyecharts import options as opts
from pyecharts.charts import Bar,Kline,Line,Grid,Tab,Candlestick
from pyecharts.globals import ThemeType
import  pyecharts.charts
import talib 
from talib import MA_Type
import pandas as pd
import datetime
import pandas as pd
import os
import json
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [14]:
allticker_path = r'D:\量化\my_team\code\data\day_data'
data_path = r'D:\量化\my_team\code\data'
industry_path = r'D:\量化\my_team\code\data\industry_data'
allticker_file = os.listdir(allticker_path)

In [58]:
df=pd.read_csv(allticker_path+'/FUBO.csv',encoding='GBK')
df.head()

,Date,Open,High,Low,Close,Volume
0,2019-03-27,10.20,10.20,9.27,9.99,1200
1,2019-03-28,9.99,9.99,9.99,9.99,300
2,2019-03-29,9.99,9.99,9.99,9.99,100
3,2019-04-01,9.49,10.00,9.49,10.00,700
4,2019-04-02,9.20,9.25,6.25,7.10,2200


In [79]:
def Data_pre(df):
    # 计算MACD
    df['DIF'], df['DEA'], df['MACD'] = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    # 计算MACD的DEA(可选)与股价直接的10日内的相关系数
    df['MACD_corr'] = df['Close'].rolling(10).corr(df.DEA)
    df['FLAG']=np.where( ( (df['MACD_corr']>0) & ( (df['MACD_corr'].shift(2)<-0.3) | (df['MACD_corr'].shift(3)<-0.3) | (df['MACD_corr'].shift(4)<-0.3))
                          & (df['MACD_corr'].shift(1)<-0.0) 
                         )
        ,1,0)
    # 计算DEA的斜率
    df['slope21'] = df['DEA'].rolling(10).apply(lambda x: np.polyfit(list(range(1,11)), x.tolist(), deg=1)[0])
    # 计算股价的斜率
    df['close_slope21'] = df['Close'].rolling(10).apply(lambda x: np.polyfit(list(range(1,11)), x.tolist(), deg=1)[0])
    # 标注做多的信号点
    df['BUY'] = np.where(
       ( (df['FLAG']==1)  &  ( 
           ( (df['slope21'].shift(1)>-0.003) & (df['close_slope21'].shift(1)<0.003) )|
          (  (df['slope21'].shift(2)>-0.003) & (df['close_slope21'].shift(2)<0.003) )
            )   
       ),1,0 )
    # 标注做空的信号点
    df['SELL'] = np.where(
        ( (df['BUY']==0) & (df['FLAG']==1))

    ,1,0)
    # 分别计算当天之后 5 ，10 ，20天的涨跌幅
    df['pct_chg_5'] = 100*(df['Close'].shift(-5)/df['Close'] - 1)
    df['pct_chg_10'] = 100*(df['Close'].shift(-10)/df['Close'] - 1)
    df['pct_chg_20'] = 100*(df['Close'].shift(-20)/df['Close'] - 1)

    # 分别计算当天之后 5 ，10 ，20天的斜率
    df['slope_5'] = df['Close'].rolling(5).apply(lambda x: np.polyfit(list(range(1,6)), x.tolist(), deg=1)[0]).shift(-5)
    df['slope_10'] = df['Close'].rolling(10).apply(lambda x: np.polyfit(list(range(1,11)), x.tolist(), deg=1)[0]).shift(-10)
    df['slope_20'] = df['Close'].rolling(20).apply(lambda x: np.polyfit(list(range(1,21)), x.tolist(), deg=1)[0]).shift(-20)
    
    return df

In [10]:
def get_signal_data(df):
    output_df = df[((df['BUY']==1) | (df['SELL']==1))]
    output_df = output_df[['Date','BUY','SELL','pct_chg_5','pct_chg_10','pct_chg_20']]
    output_df.reset_index(drop=True,inplace=True)
    output_df.dropna(inplace=True)
    del output_df['SELL']
    output_df.rename(columns={'BUY':'trade_type'},inplace = True)
    return output_df

In [55]:
df = Data_pre(df)
output_df = get_signal_data(df)

In [56]:
output_df

,Date,trade_type,pct_chg_5,pct_chg_10,pct_chg_20
0,2010-09-21,1,0.451492,6.803681,7.504265
1,2010-12-01,0,-0.821680,-2.251951,-1.080336
2,2011-02-10,0,-0.577992,-0.495414,-1.403776
3,2011-09-15,1,-8.706483,-3.031741,-1.228236
4,2012-01-27,0,2.401077,1.128229,1.479056
5,2012-02-13,0,1.162321,0.414149,-0.721453
6,2012-02-28,0,-3.459301,-1.130918,-0.465639
7,2013-06-19,0,-1.467002,0.625891,5.261586
8,2013-08-12,0,0.460499,1.237547,3.770152
9,2013-10-18,0,7.076380,8.578201,11.472371


In [77]:
all_output_data = pd.DataFrame()
for code in allticker_file[:100]:
    print(code)
    df = pd.read_csv(allticker_path+'/'+code,encoding='GBK')
    if(len(df)>200):
        df = Data_pre(df)
        output_df = get_signal_data(df)
        all_output_data = all_output_data.append(output_df)
all_output_data

A.CSV
AA.CSV
AAA.CSV
AAAU.CSV
AABA.CSV
AAC.CSV
AACG.CSV
AACOU.CSV
AACQ.CSV
AACQU.CSV
AACQW.CSV
AADR.CSV
AAGH.CSV
AAIC-PC.CSV
AAIC.CSV
AAL.CSV
AAMC.CSV
AAME.CSV
AAN.CSV
AAOI.CSV
AAON.CSV
AAP.CSV
AAPL.CSV
AASP.CSV
AAT.CSV
AAU.CSV
AAWW.CSV
AAXJ.CSV
AAXN.CSV
AB.CSV
ABAX.CSV
ABB.CSV
ABBV.CSV
ABC.CSV
ABCB.CSV
ABCL.CSV
ABCM.CSV
ABCP.CSV
ABDC.CSV
ABE.CSV
ABEO.CSV
ABEOW.CSV
ABEQ.CSV
ABEV.CSV
ABG.CSV
ABGI.CSV
ABIL.CSV
ABILF.CSV
ABIO.CSV
ABM.CSV
ABMC.CSV
ABMD.CSV
ABML.CSV
ABMT.CSV
ABNB.CSV
ABQQ.CSV
ABR-PB.CSV
ABR-PC.CSV
ABR.CSV
ABST.CSV
ABT.CSV
ABTI.CSV
ABTX.CSV
ABUS.CSV
ABVC.CSV
ABVN.CSV
ABX.CSV
AC.CSV
ACA.CSV
ACAC.CSV
ACACU.CSV
ACACW.CSV
ACAD.CSV
ACAHU.CSV
ACAM.CSV
ACAMU.CSV
ACAMW.CSV
ACAN.CSV
ACB.CSV
ACBAU.CSV
ACBI.CSV
ACBM.CSV
ACC.CSV
ACCA.CSV
ACCD.CSV
ACCO.CSV
ACEL.CSV
ACER.CSV
ACET.CSV
ACEV.CSV
ACEVU.CSV
ACEVW.CSV
ACFN.CSV
ACGL.CSV
ACGLO.CSV
ACGLP.CSV
ACH.CSV
ACHC.CSV
ACHL.CSV
ACHN.CSV


,Date,trade_type,pct_chg_5,pct_chg_10,pct_chg_20
0,2010-04-01,0,-0.926182,2.083924,4.949347
1,2010-07-22,1,-1.984692,-0.905275,-0.626770
2,2010-11-04,0,-0.722393,0.666886,4.140054
3,2011-01-06,0,3.592069,1.518794,3.640293
4,2011-11-16,0,-12.908237,-1.969938,-13.297056
...,...,...,...,...,...
21,2019-05-23,0,-0.062036,5.179895,7.692306
22,2019-09-06,1,18.144863,21.064298,10.347381
23,2020-08-17,0,-1.308899,1.145289,-1.047119
24,2020-12-01,0,3.644590,14.221061,19.818961


In [18]:
# all_output_data.to_csv(data_path+'/ALL_LOOK.csv',index=False,encoding='GBK')

In [20]:
# print('总数据量:{}'.format(len(all_output_data)))

总数据量:791055


In [30]:
def report(output_df):
    sum_data = len(output_df)
    sum_up = len(output_df[(output_df['trade_type']==1)])
    sum_down = len(output_df[(output_df['trade_type']==0)])
    
    buy_up_5 = len(output_df[((output_df['trade_type']==1) & (output_df['pct_chg_5']>0))])
    buy_up_10 = len(output_df[((output_df['trade_type']==1) & (output_df['pct_chg_10']>0))])
    buy_up_20 = len(output_df[((output_df['trade_type']==1) & (output_df['pct_chg_20']>0))])
                    
    sell_dwon_5 = len(output_df[((output_df['trade_type']==0) & (output_df['pct_chg_5']<0))])
    sell_dwon_10 = len(output_df[((output_df['trade_type']==0) & (output_df['pct_chg_10']<0))])
    sell_dwon_20 = len(output_df[((output_df['trade_type']==0) & (output_df['pct_chg_20']<0))])
                    
    print('总数据量:{}'.format(sum_data))
    print('给出做多信号数据量:{}'.format(sum_up))      
    print('给出做空信号数据量:{}'.format(sum_down))
    
    print('在判断为上涨信号时，5天后为上涨的概率:{}'.format(buy_up_5/sum_up))
    print('在判断为上涨信号时，10天后为上涨的概率:{}'.format(buy_up_10/sum_up))
    print('在判断为上涨信号时，20天后为上涨的概率:{}'.format(buy_up_20/sum_up))
                    
    print('在判断为下跌信号时，5天后为下跌的概率:{}'.format(sell_dwon_5/sum_down))
    print('在判断为下跌信号时，10天后为下跌的概率:{}'.format(sell_dwon_10/sum_down))
    print('在判断为下跌信号时，20天后为下跌的概率:{}'.format(sell_dwon_20/sum_down))
    
    print('5天的总体胜率:{}'.format((buy_up_5+sell_dwon_5)/sum_data))
    print('10天的总体胜率:{}'.format((buy_up_10+sell_dwon_10)/sum_data))
    print('20天的总体胜率:{}'.format((buy_up_20+sell_dwon_20)/sum_data))

In [78]:
report(all_output_data)

总数据量:1453
给出做多信号数据量:757
给出做空信号数据量:696
在判断为上涨信号时，5天后为上涨的概率:0.4147952443857332
在判断为上涨信号时，10天后为上涨的概率:0.4557463672391017
在判断为上涨信号时，20天后为上涨的概率:0.5151915455746368
在判断为下跌信号时，5天后为下跌的概率:0.4525862068965517
在判断为下跌信号时，10天后为下跌的概率:0.45977011494252873
在判断为下跌信号时，20天后为下跌的概率:0.4410919540229885
5天的总体胜率:0.4328974535443909
10天的总体胜率:0.4576737783895389
20天的总体胜率:0.4796971782518926


In [80]:
#  测试
df=pd.read_csv(allticker_path+'/AAPL.csv',encoding='GBK')
df = Data_pre(df)
output_df = get_signal_data(df)
report(output_df)

总数据量:130
给出做多信号数据量:44
给出做空信号数据量:86
在判断为上涨信号时，5天后为上涨的概率:0.5909090909090909
在判断为上涨信号时，10天后为上涨的概率:0.7272727272727273
在判断为上涨信号时，20天后为上涨的概率:0.7045454545454546
在判断为下跌信号时，5天后为下跌的概率:0.4418604651162791
在判断为下跌信号时，10天后为下跌的概率:0.4883720930232558
在判断为下跌信号时，20天后为下跌的概率:0.37209302325581395
5天的总体胜率:0.49230769230769234
10天的总体胜率:0.5692307692307692
20天的总体胜率:0.4846153846153846
